# Laboratorio 9

### Integrantes:

 - BRANDON JAVIER REYES MORALES
 - CARLOS ALBERTO VALLADARES GUERRA 
 - JUAN PABLO SOLIS ALBIZUREZ

### **Task 1: Teoria**

1. Diga cuál es la diferencia entre Modelos de Markov y Hidden Markov Models
- En los modelos de markov se representan sistemas donde el estado actual depende únicamente del estado anterior. Estos modelos asumen que todos los estados son observables. Por otro lado, los Hidden Markov Models suponen que los estados verdaderos del sistema son ocultos y no se pueden observar directamente. En su lugar, se observan señales o emisiones que dependen probabilísticamente de estos estados ocultos.

2. Investigue qué son los factorial HMM (Hidden Markov Models)
- Los factorial Hidden Markov Models  son una generalización de los Hidden Markov Models tradicionales. En lugar de tener una sola cadena de estados ocultos, los FHMM utilizan múltiples cadenas ocultas que evolucionan de forma paralela e independiente. Cada cadena representa una parte diferente del sistema oculto, y las observaciones se generan combinando la información de todas las cadenas. Esto permite modelar fenómenos más complejos, ya que se puede capturar la influencia simultánea de múltiples factores ocultos.

3. Especifique en sus propias palabras el algoritmo Forward-Backward para HMM
- El algoritmo Forward-Backward es una técnica utilizada en los HMM para calcular la probabilidad de que el sistema esté en un cierto estado en un momento dado, considerando toda la secuencia de observaciones. Este algoritmo tiene dos fases: la fase "Forward" calcula, hacia adelante en el tiempo, la probabilidad acumulada de haber alcanzado cada estado, dada la secuencia de observaciones hasta ese punto. La fase "Backward" hace el cálculo inverso: determina la probabilidad de observar la secuencia restante a partir de un estado específico, hacia atrás en el tiempo. 

4. En el algoritmo de Forward-Backward, ¿por qué es necesario el paso de Backward?
- El paso de Backward es necesario porque nos permite tener en cuenta no solo el pasado, sino también el futuro en nuestras estimaciones. Si solo usamos el paso Forward, solo consideraríamos la información hasta el momento actual, lo cual puede ser insuficiente para tomar decisiones precisas. Al incluir el paso Backward, integramos también las observaciones futuras, lo cual mejora significativamente la precisión de la estimación de los estados ocultos. 